In [1]:
import re
import json
import pickle
import os
import sys
import logging
import torch
from transformer_lens import HookedTransformer
import plotly.io as pio
import pandas as pd
from ipywidgets import interact, IntSlider
from process_tiny_stories_data import load_tinystories_validation_prompts, load_tinystories_tokens
from typing import Literal
from transformer_lens.utils import test_prompt

pio.renderers.default = "notebook_connected"
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

sys.path.append('../')  # Add the parent directory to the system path

import utils.haystack_utils as haystack_utils
from sparse_coding.train_autoencoder import AutoEncoder
from utils.autoencoder_utils import custom_forward, AutoEncoderConfig, evaluate_autoencoder_reconstruction, get_encoder_feature_frequencies, load_encoder, generate_with_encoder
import utils.haystack_utils as haystack_utils

%reload_ext autoreload
%autoreload 2

In [5]:
model_name = "tiny-stories-33M"
model = HookedTransformer.from_pretrained(
    model_name,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    device=device,
)

(…)TinyStories-33M/resolve/main/config.json:   0%|          | 0.00/968 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/291M [00:00<?, ?B/s]

(…)s-33M/resolve/main/tokenizer_config.json:   0%|          | 0.00/722 [00:00<?, ?B/s]

(…)/TinyStories-33M/resolve/main/vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

(…)/TinyStories-33M/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)yStories-33M/resolve/main/tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

(…)33M/resolve/main/special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

Loaded pretrained model tiny-stories-33M into HookedTransformer


In [10]:
l0_encoder, l0_config = load_encoder('2_silvery_smoke', model_name, model)
l1_encoder, l1_config = load_encoder('2_soft_monkey', model_name, model)
prompts = load_tinystories_validation_prompts()

In [12]:
torch.autograd.set_grad_enabled(True)
torch.set_grad_enabled(True)

# For single prompt: compare the derivative of the L1 feature wrt L0 mlp input to layer, to linear map (assuming GELU is identity)
prompt = prompts[0]

def get_encode_activations_hooks(encoder, config):
    def encode_activations_hook(value, hook):
        value = value.squeeze(0)
        _, x_reconstruct, _, _, _ = encoder(value)
        print(value.grad_fn)
        print(x_reconstruct.grad_fn)
        return x_reconstruct.unsqueeze(0)
    reconstruct_hook_label = f'blocks.{config.layer}.{config.act_name}'
    return [(reconstruct_hook_label, encode_activations_hook)]

grads = []
grad_fns = []
def save_grad(value, hook):
    grads.append(value.grad)
    grad_fns.append(value.grad_fn)

save_grads = [('blocks.0.mlp.hook_post', save_grad), ('blocks.1.mlp.hook_post', save_grad)]
with model.hooks(get_encode_activations_hooks(l0_encoder, l0_config) + save_grads): #, get_encode_activations_hooks(l1_encoder, l1_config)):
    loss = model(prompt, return_type='loss')

print(grads[0])
print(grad_fns[0])
loss.backward()

# The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. 
# Its .grad attribute won't be populated during autograd.backward(). 
# If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. 
# If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. 
# See github.com/pytorch/pytorch/pull/30531 for more informations.

None


/tmp/ipykernel_1926/386258236.py:20: UserWarning:

The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)

